<a href="https://colab.research.google.com/github/brianferrell787/Tensorboard-graph-problem-fixed-for-Google-Colab/blob/master/ColabTensorboardissuefixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fixing Tensorboard issue on Google Colab

Copyright 2017 The TensorFlow Authors.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Upload and clean data
(other preprocessing steps just for my specific data)

In [1]:
#Upload data that has examples that will be used for attention layer output(only a few examples were pulled though)
import pandas as pd
val_df = pd.read_excel('training_df.xlsx' )
val_df =val_df.fillna("")
val_df

,AimsGoal,Hypothesis,Study Design,Background,Class
0,"through message testing , all investigators an...",this study examines rape prevention messages f...,this project consists of a type of study group...,more than individuals who are 50 years of age ...,1
1,"rfc message testing , all investigators and me...",this study includes revised rfc messages for t...,this study consists of a series of focus group...,recognizing than individuals who are 50 years ...,1
2,wv physician professionals health survey [ wvm...,what is the environmental impact of substance ...,using self - identified healthcare expenses fr...,recognizing than individuals who are 50 years ...,1
3,wv healthcare professionals assistance program...,what is the adverse effects of drug misuse on ...,when self - identified patient data from the s...,in the realm of upper - secondary educational ...,1
4,wv rehabilitation mental health program [ wvmp...,what is the psychological impact of drug misus...,using pre - identified patient data from the w...,in the realm of post - secondary educational i...,1
...,...,...,...,...,...
2622,lacking capacity for children ' s participatio...,"this study will provide an interpretivist , qu...",study design : this is a qualitative interpret...,despite legislative mandates supporting childr...,5
2623,having partners interact with richmond health ...,this article will contain lists of prominent i...,qualitative analysis will be used in this stud...,pressured by an aging and increasingly complex...,5
2624,in partner interaction with the health and wel...,this page will contain lists of notable person...,qualitative data will be used in this study . ...,pressured by an increasingly and increasingly ...,5
2625,lgbtq youth - serving organizations : national...,what are the economic and community - based im...,over the previous year ( beginning in the fall...,"significance beginning in the 1980s , two tren...",5


In [2]:
#This step doesn't need to be done if all of your columns are already put together into one. Ours were seperate. 
val_df['New2'] = val_df['AimsGoal'] + val_df['Hypothesis'] + val_df['Background'] + val_df['Study Design']
val_df 


,AimsGoal,Hypothesis,Study Design,Background,Class,New2
0,"through message testing , all investigators an...",this study examines rape prevention messages f...,this project consists of a type of study group...,more than individuals who are 50 years of age ...,1,"through message testing , all investigators an..."
1,"rfc message testing , all investigators and me...",this study includes revised rfc messages for t...,this study consists of a series of focus group...,recognizing than individuals who are 50 years ...,1,"rfc message testing , all investigators and me..."
2,wv physician professionals health survey [ wvm...,what is the environmental impact of substance ...,using self - identified healthcare expenses fr...,recognizing than individuals who are 50 years ...,1,wv physician professionals health survey [ wvm...
3,wv healthcare professionals assistance program...,what is the adverse effects of drug misuse on ...,when self - identified patient data from the s...,in the realm of upper - secondary educational ...,1,wv healthcare professionals assistance program...
4,wv rehabilitation mental health program [ wvmp...,what is the psychological impact of drug misus...,using pre - identified patient data from the w...,in the realm of post - secondary educational i...,1,wv rehabilitation mental health program [ wvmp...
...,...,...,...,...,...,...
2622,lacking capacity for children ' s participatio...,"this study will provide an interpretivist , qu...",study design : this is a qualitative interpret...,despite legislative mandates supporting childr...,5,lacking capacity for children ' s participatio...
2623,having partners interact with richmond health ...,this article will contain lists of prominent i...,qualitative analysis will be used in this stud...,pressured by an aging and increasingly complex...,5,having partners interact with richmond health ...
2624,in partner interaction with the health and wel...,this page will contain lists of notable person...,qualitative data will be used in this study . ...,pressured by an increasingly and increasingly ...,5,in partner interaction with the health and wel...
2625,lgbtq youth - serving organizations : national...,what are the economic and community - based im...,over the previous year ( beginning in the fall...,"significance beginning in the 1980s , two tren...",5,lgbtq youth - serving organizations : national...


In [3]:
#Import packages and create our cleaning function that will get rid of unnecessary noise in our dataset + lemmatizing 
import nltk
import re  
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import string
nltk.download('stopwords')
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    nltk.download('stopwords')
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", "  ", text)
    text = re.sub(r"=", "  ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", "  ", text)
    text = re.sub(r";", "  ", text)
    text = re.sub(r"http", "  ", text)
    text = re.sub(r"www", "  ", text)
    text = re.sub(r"-", "  ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\d+", " ", text)

    ## Lemmatize
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemma_words)

    return text


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Clean the data 
val_df['New2'] =val_df['New2'].astype(str)
valtext = val_df['New2'].map(lambda x: clean_text(x))

Streaming output truncated to the last 5000 lines.
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data.

In [7]:
#Tokenize, sequence, padding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 19000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(valtext)
sequences = tokenizer.texts_to_sequences(valtext)
xtrain = pad_sequences(sequences, maxlen=10137)



## Setup

In [8]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [9]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.3.0


In [10]:
import tensorboard
tensorboard.__version__

'2.3.0'

In [11]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Define a Keras model

In this example, I have to give my uploaded model the attention layer I used

In [12]:
from keras.engine.topology import Layer
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [19]:
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope
model = load_model('whole69.mdl_wts.hdf5',custom_objects={'attention': attention},compile=True)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 10137)]      0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 10137, 100)   1900000     input_9[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_9 (SpatialDro (None, 10137, 100)   0           embedding_9[0][0]                
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 10137, 128)   84992       spatial_dropout1d_9[0][0]        
____________________________________________________________________________________________

## Train the model and log data

Before training, define the [Keras TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard), specifying the log directory. By passing this callback to Model.fit(), you ensure that graph data is logged for visualization in TensorBoard.

In [18]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    xtrain,
    y_train, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
33/33 [==============================] - 94s 3s/step - loss: 0.2454 - accuracy: 0.0000e+00
Epoch 2/5
33/33 [==============================] - 92s 3s/step - loss: 0.0826 - accuracy: 0.0000e+00
Epoch 3/5
33/33 [==============================] - 92s 3s/step - loss: 0.0409 - accuracy: 0.0000e+00
Epoch 4/5
33/33 [==============================] - 92s 3s/step - loss: 0.0307 - accuracy: 0.0000e+00
Epoch 5/5
33/33 [==============================] - 92s 3s/step - loss: 0.0166 - accuracy: 0.0000e+00


## This might not work for you which is why I am here to fix it

Start TensorBoard and wait a few seconds for the UI to load. Select the Graphs dashboard by tapping “Graphs” at the top. 

In [15]:
%tensorboard --logdir logs

<IPython.core.display.Javascript object>